In [1]:
import pandas as pd
import numpy as np

In [116]:
train_data = pd.read_csv("Predict_NoShow_Train.csv")
private_test_data = pd.read_csv("Predict_NoShow_PrivateTest_WithoutLabels.csv")
public_test_data = pd.read_csv("Predict_NoShow_PublicTest_WithoutLabels.csv")

In [3]:
train_data.head()

,ID,Age,Gender,DateAppointmentWasMade,DateOfAppointment,DaysUntilAppointment,Diabetes,Alcoholism,Hypertension,Handicapped,Smoker,Scholarship,Tuberculosis,RemindedViaSMS,DayOfTheWeek,Status
0,904656,15,M,2014-01-29,2014-02-10,12,0,0,0,0,0,0,0,1,Monday,Show-Up
1,174414,2,M,2014-07-02,2014-07-09,7,0,0,0,0,0,0,0,1,Wednesday,Show-Up
2,416498,46,M,2015-02-25,2015-02-27,2,0,0,0,0,0,0,0,0,Friday,Show-Up
3,662378,16,F,2014-10-13,2014-10-22,9,0,0,0,0,0,1,0,0,Wednesday,Show-Up
4,497621,59,M,2014-09-30,2014-10-02,2,1,0,1,0,0,0,0,0,Thursday,Show-Up


In [112]:
train_data.shape, private_test_data.shape, public_test_data.shape

((180000, 16), (60000, 15), (60000, 15))

In [5]:
train_data.groupby("Status").size()

Status
No-Show     53993
Show-Up    126007
dtype: int64

Unbalanced Data

In [117]:
show_train = train_data[train_data["Status"]=="Show-Up"]
no_train = train_data[train_data["Status"]=="No-Show"]

In [118]:
show_train.shape, no_train.shape

((126007, 16), (53993, 16))

In [13]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [23]:
show_train = show_train.sample(n=53993, random_state=2018)

In [119]:
x_train = pd.concat([show_train, no_train])

In [120]:
x_train = x_train.sample(frac=1, random_state=2018)

In [121]:
x_train.Gender = x_train.Gender.apply(lambda x: 1 if x == "M" else 0)

In [122]:
x_train.head(3)

,ID,Age,Gender,DateAppointmentWasMade,DateOfAppointment,DaysUntilAppointment,Diabetes,Alcoholism,Hypertension,Handicapped,Smoker,Scholarship,Tuberculosis,RemindedViaSMS,DayOfTheWeek,Status
82389,166916,30,1,2015-02-04,2015-02-09,5,0,0,0,0,0,0,0,1,Monday,Show-Up
20531,720166,37,0,2015-01-26,2015-01-28,2,0,0,0,0,0,0,0,0,Wednesday,Show-Up
1131,301372,72,0,2014-11-12,2014-11-21,9,1,0,1,0,0,0,0,0,Friday,Show-Up


In [123]:
x_train.drop(["ID", "DateAppointmentWasMade", "DateOfAppointment"], axis=1, inplace=True)

In [124]:
day_dict = {"Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4, "Friday": 5,\
    "Saturday": 6, "Sunday": 7}
x_train.DayOfTheWeek = x_train.DayOfTheWeek.apply(lambda x: day_dict[x])

In [125]:
x_train.head(4)

,Age,Gender,DaysUntilAppointment,Diabetes,Alcoholism,Hypertension,Handicapped,Smoker,Scholarship,Tuberculosis,RemindedViaSMS,DayOfTheWeek,Status
82389,30,1,5,0,0,0,0,0,0,0,1,1,Show-Up
20531,37,0,2,0,0,0,0,0,0,0,0,3,Show-Up
1131,72,0,9,1,0,1,0,0,0,0,0,5,Show-Up
84591,5,1,3,0,0,0,0,0,1,0,0,5,Show-Up


In [82]:
y = x_train.pop("Status")

In [83]:
target = [1 if x == "No-Show" else 0 for x in y]

Train

In [130]:
model = RandomForestClassifier()

In [132]:
cross_val_score(model, x_train_dua, target, cv=5)

array([0.6523094 , 0.65156891, 0.64566492, 0.6491515 , 0.64972693])

In [133]:
model.fit(x_train_dua, target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [134]:
model.feature_importances_, x_train_dua.columns

(array([0.52315327, 0.02000125, 0.27810455, 0.00842108, 0.00626041,
        0.01160181, 0.01149229, 0.00973641, 0.00916425, 0.00073499,
        0.02182447, 0.09950521]),
 Index(['Age', 'Gender', 'DaysUntilAppointment', 'Diabetes', 'Alcoholism',
        'Hypertension', 'Handicapped', 'Smoker', 'Scholarship', 'Tuberculosis',
        'RemindedViaSMS', 'DayOfTheWeek'],
       dtype='object'))

'Age', 'DaysUntilAppointment', 'DayOfTheWeek'

In [89]:
0.45406121+0.36955829+0.08425527

0.9078747699999999

In [106]:
sum(x_train.groupby("DaysUntilAppointment").size()[0:30])

162051

In [126]:
x_train_dua = x_train[x_train.DaysUntilAppointment.apply(lambda x: True if x <= 30 else False)]

In [127]:
x_train_dua.head(3)

,Age,Gender,DaysUntilAppointment,Diabetes,Alcoholism,Hypertension,Handicapped,Smoker,Scholarship,Tuberculosis,RemindedViaSMS,DayOfTheWeek,Status
82389,30,1,5,0,0,0,0,0,0,0,1,1,Show-Up
20531,37,0,2,0,0,0,0,0,0,0,0,3,Show-Up
1131,72,0,9,1,0,1,0,0,0,0,0,5,Show-Up


In [128]:
y = x_train_dua.pop("Status")
target = [1 if x == "No-Show" else 0 for x in y]

In [40]:
from sklearn.metrics import accuracy_score

In [135]:
accuracy_score(model.predict(x_train_dua), target)

0.806320232519392

In [137]:
id_public = public_test_data.ID
public_test_data = public_test_data.drop(["ID", "DateAppointmentWasMade", "DateOfAppointment"], axis=1)
# Age minmax
# age_max, age_min = public_test_data.Age.max(), public_test_data.Age.min()
# public_test_data.Age = public_test_data.Age.apply(lambda x: float(x - age_min)/ (age_max - age_min))
# Gender to numeric
gender_dict = {"M": 0, "F": 1}
gender = [gender_dict[x] for x in public_test_data["Gender"]]
public_test_data["Gender"] = pd.Series(gender)
# DayOfTheWeek to numeric
day_dict = {"Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4, "Friday": 5,\
    "Saturday": 6, "Sunday": 7}
day = [day_dict[x] for x in public_test_data["DayOfTheWeek"]]
public_test_data["DayOfTheWeek"] = pd.Series(day)

In [138]:
prob=model.predict_proba(public_test_data)[:,1]

In [67]:
label = [1 if x > 0.7 else 0 for x in prob]

In [68]:
from collections import OrderedDict

In [69]:
public = pd.DataFrame(OrderedDict({"ID": id_public, "prob": prob, "label": label}))
public.to_csv("public.csv", header=None, index=None)

test

In [53]:
from sklearn.metrics import roc_curve, auc

In [54]:
fpr, tpr, thresholds =roc_curve(target, model.predict(x_train))

In [61]:
def cutoff_youdens_j(fpr,tpr,thresholds):
    j_scores = tpr-fpr
    j_ordered = sorted(zip(j_scores,thresholds))
    return j_ordered[-1][1]

In [62]:
cutoff_youdens_j(fpr,tpr, thresholds)

1

In [60]:
threshold

[1]

In [56]:
auc(fpr, tpr)

0.7959457707480599

In [63]:
from sklearn.metrics import confusion_matrix

In [64]:
confusion_matrix(target, model.predict(x_train))

array([[43935, 10058],
       [11977, 42016]], dtype=int64)